### 1. Import thư viện

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression, DecisionTreeRegressor, RandomForestRegressor
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import countDistinct
from pyspark.sql import DataFrame
from pyspark.sql.functions import when, col
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import SparkSession, functions as F
from xgboost.spark import SparkXGBRegressor
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

spark = (
    SparkSession.builder
    .appName("RegressionModels")
    .getOrCreate()
)

25/10/27 07:49:55 WARN Utils: Your hostname, soaz resolves to a loopback address: 127.0.1.1; using 192.168.1.18 instead (on interface wlp0s20f3)
25/10/27 07:49:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/27 07:49:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### 2. Đọc dữ liệu

In [2]:

df = spark.read.csv('../../data/cleaned_data/data.csv', inferSchema= True, header= True)

print("Dữ liệu ban đầu:")
df.show(5, truncate=False)
df.printSchema()


Dữ liệu ban đầu:
+--------+---------------------+------------+------+---+----+----+------+
|image_id|street               |citi        |n_citi|bed|bath|sqft|price |
+--------+---------------------+------------+------+---+----+----+------+
|1       |124 C Street W       |Brawley, CA |48    |3  |2   |713 |228500|
|2       |2304 Clark Road      |Imperial, CA|152   |3  |1   |800 |273950|
|3       |755 Brawley Avenue   |Brawley, CA |48    |3  |1   |1082|350000|
|4       |2207 R Carrillo Court|Calexico, CA|55    |4  |3   |2547|385100|
|6       |1100 CAMILIA Street  |Calexico, CA|55    |4  |3   |2769|415000|
+--------+---------------------+------------+------+---+----+----+------+
only showing top 5 rows

root
 |-- image_id: integer (nullable = true)
 |-- street: string (nullable = true)
 |-- citi: string (nullable = true)
 |-- n_citi: integer (nullable = true)
 |-- bed: integer (nullable = true)
 |-- bath: integer (nullable = true)
 |-- sqft: integer (nullable = true)
 |-- price: integer (nu

### 3. Tiền xử lý dữ liệu và chuẩn bị features

#### Xử lý cột street

In [3]:

df = df.withColumn("street", regexp_replace("street", "^[^ ]+ ", ""))

In [4]:
df.groupBy("street").count().orderBy(col("count").desc()).show()



+------------------+-----+
|            street|count|
+------------------+-----+
|      not provided|   19|
|      Calle Aragon|   15|
|   Avenida Majorca|   14|
|   Avenida Sevilla|   10|
|    Woodland Drive|   10|
|     Walnut Street|    8|
|        9th Street|    8|
|Country Club Drive|    8|
|   Del Monte Drive|    8|
|  Avenida Castilla|    8|
| W San Marcos Blvd|    8|
|     Acacia Avenue|    7|
|   N Euclid Avenue|    7|
|       Renault Way|    7|
|    Palomar Avenue|    7|
|        4th Street|    7|
|   El Dorado Drive|    7|
|       Park Avenue|    7|
|   Magnolia Avenue|    6|
|        Ash Street|    6|
+------------------+-----+
only showing top 20 rows



In [5]:

df.select(countDistinct("street")).show()

+----------------------+
|count(DISTINCT street)|
+----------------------+
|                 10378|
+----------------------+



In [6]:


street_freq = df.groupBy("street").count().withColumnRenamed("count", "street_frequency")

df = df.join(street_freq, on="street", how="left")

df = df.withColumn("street", col("street_frequency")).drop("street_frequency")

#### Xử lý cột citi

In [7]:
df.groupBy("citi").count().orderBy(col("count").desc()).show()

+--------------------+-----+
|                citi|count|
+--------------------+-----+
|       San Diego, CA|  579|
|     Los Angeles, CA|  307|
|       Lancaster, CA|  270|
|       La Quinta, CA|  227|
|       Riverside, CA|  221|
|          Corona, CA|  208|
|       Escondido, CA|  198|
|        Palmdale, CA|  195|
|         Fontana, CA|  182|
|        Big Bear, CA|  175|
|    Palm Springs, CA|  172|
|Rancho Cucamonga, CA|  164|
|       Oceanside, CA|  161|
|        Murrieta, CA|  148|
|     Palm Desert, CA|  145|
|     Paso Robles, CA|  141|
|  San Bernardino, CA|  138|
|  Lake Arrowhead, CA|  137|
|   Rancho Mirage, CA|  133|
|        Temecula, CA|  132|
+--------------------+-----+
only showing top 20 rows



In [8]:
df.select(countDistinct("citi")).show()

+--------------------+
|count(DISTINCT citi)|
+--------------------+
|                 415|
+--------------------+



In [9]:

def spark_get_dummies(df: DataFrame, column: str) -> DataFrame:
    unique_values = [row[0] for row in df.select(column).distinct().collect()]
    for val in unique_values:
        df = df.withColumn(f"{column}_{val}", when(col(column) == val, 1).otherwise(0))
    return df.drop(column)


In [10]:
df = spark_get_dummies(df, "citi")


In [11]:
cols = [c for c in df.columns if c != "price"] + ["price"]
df = df.select(*cols)  
                           

In [12]:
df.show(5, truncate=False)
df.printSchema()

25/10/27 07:50:29 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------+--------+------+---+----+----+---------------+---------------+----------------+----------------+----------------------+------------------------+--------------+---------------------+--------------------+-----------------+---------------------+-----------------+---------------------+--------------------+------------------+-----------------------+-------------------+------------------------+-------------------+------------------+---------------------+------------------------+----------------+---------------+----------------+-------------------+-------------------+--------------------+------------------------+-------------------+--------------+----------------+------------------+---------------------+-------------------+-----------------+------------------+-----------------+----------------+--------------------+---------------+-----------------+----------------------+-------------------+---------------------+-------------------------+----------------+-----------------------------+-

#### Tách đặc trưng và nhãn

In [13]:

columns = df.columns
feature_cols = columns[:-1] 
label_col = columns[-1]  

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
df_transformed = assembler.transform(df)

df_transformed.show(5, truncate=False)


+------+--------+------+---+----+----+---------------+---------------+----------------+----------------+----------------------+------------------------+--------------+---------------------+--------------------+-----------------+---------------------+-----------------+---------------------+--------------------+------------------+-----------------------+-------------------+------------------------+-------------------+------------------+---------------------+------------------------+----------------+---------------+----------------+-------------------+-------------------+--------------------+------------------------+-------------------+--------------+----------------+------------------+---------------------+-------------------+-----------------+------------------+-----------------+----------------+--------------------+---------------+-----------------+----------------------+-------------------+---------------------+-------------------------+----------------+-----------------------------+-

### Chia train, valid và test

In [14]:

# --- 2. Chia train / valid / test --- 
train_data, valid_data, test_data = df_transformed.randomSplit([0.7, 0.15, 0.15], seed=42)

# --- 3. Hàm đánh giá ---
def evaluate(predictions, label_col):
    evaluator_rmse = RegressionEvaluator(labelCol=label_col, predictionCol="prediction", metricName="rmse")
    evaluator_r2 = RegressionEvaluator(labelCol=label_col, predictionCol="prediction", metricName="r2")
    rmse = evaluator_rmse.evaluate(predictions)
    r2 = evaluator_r2.evaluate(predictions)
    print(f"RMSE = {rmse:.4f} | R² = {r2:.4f}")
    return rmse, r2

### 4. Huấn luyện mô hình

#### 4.1. Linear Regression

In [15]:

lr = LinearRegression(featuresCol="features", labelCol=label_col)

paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.01, 0.1, 0.3, 0.5])
             .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])
             .addGrid(lr.maxIter, [100, 300, 500])
             .build())

evaluator = RegressionEvaluator(labelCol=label_col, predictionCol="prediction", metricName="rmse")

crossval = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=3)

print("\nTraining Linear Regression...")
cvModel = crossval.fit(train_data)

bestModel_lr = cvModel.bestModel

pred_lr = bestModel_lr.transform(valid_data)

evaluate(pred_lr, label_col)

print(f"Best Parameters:")
print(f"  - regParam: {bestModel_lr._java_obj.getRegParam()}")
print(f"  - elasticNetParam: {bestModel_lr._java_obj.getElasticNetParam()}")
print(f"  - maxIter: {bestModel_lr._java_obj.getMaxIter()}")



Training Linear Regression...


25/10/27 07:51:02 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/10/27 07:51:02 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
25/10/27 07:51:02 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


RMSE = 208214.4111 | R² = 0.7110
Best Parameters:
  - regParam: 0.01
  - elasticNetParam: 1.0
  - maxIter: 100


#### 4.2. Decision Tree

In [16]:

print("\nTraining Decision Tree...")

dt = DecisionTreeRegressor(featuresCol="features", labelCol=label_col)

paramGrid = (ParamGridBuilder()
             .addGrid(dt.maxDepth, [5, 10, 15, 20])
             .addGrid(dt.minInstancesPerNode, [1, 2, 4])
             .addGrid(dt.minInfoGain, [0.0, 0.01, 0.05])
             .build())

evaluator = RegressionEvaluator(labelCol=label_col, predictionCol="prediction", metricName="rmse")

crossval = CrossValidator(estimator=dt,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=3,    
                          parallelism=2)  

cvModel = crossval.fit(train_data)
bestModel_dt = cvModel.bestModel

pred_dt = bestModel_dt.transform(valid_data)

dt_rmse, dt_r2 = evaluate(pred_dt, label_col)

print(f"Best Parameters:")
print(f"  - maxDepth: {bestModel_dt.getMaxDepth()}")
print(f"  - minInstancesPerNode: {bestModel_dt.getMinInstancesPerNode()}")
print(f"  - minInfoGain: {bestModel_dt.getMinInfoGain()}")
print(f"RMSE: {dt_rmse:.4f}")
print(f"R²: {dt_r2:.4f}")



Training Decision Tree...


25/10/27 07:56:24 WARN DAGScheduler: Broadcasting large task binary with size 1024.9 KiB
25/10/27 07:56:24 WARN DAGScheduler: Broadcasting large task binary with size 1115.9 KiB
25/10/27 07:56:24 WARN DAGScheduler: Broadcasting large task binary with size 1206.2 KiB
25/10/27 07:56:24 WARN DAGScheduler: Broadcasting large task binary with size 1290.5 KiB
25/10/27 07:56:25 WARN DAGScheduler: Broadcasting large task binary with size 1365.9 KiB
25/10/27 07:56:25 WARN DAGScheduler: Broadcasting large task binary with size 1014.0 KiB
25/10/27 07:56:26 WARN DAGScheduler: Broadcasting large task binary with size 1024.9 KiB
25/10/27 07:56:26 WARN DAGScheduler: Broadcasting large task binary with size 1115.8 KiB
25/10/27 07:56:27 WARN DAGScheduler: Broadcasting large task binary with size 1206.1 KiB
25/10/27 07:56:27 WARN DAGScheduler: Broadcasting large task binary with size 1290.4 KiB
25/10/27 07:56:27 WARN DAGScheduler: Broadcasting large task binary with size 1365.9 KiB
25/10/27 07:56:28 WAR

RMSE = 191144.2677 | R² = 0.7564
Best Parameters:
  - maxDepth: 15
  - minInstancesPerNode: 4
  - minInfoGain: 0.0
RMSE: 191144.2677
R²: 0.7564


#### 4.3. Random Forest

In [17]:
print("\nTraining Random Forest...")

rf = RandomForestRegressor(
    featuresCol="features",
    labelCol=label_col
)

paramGrid = (ParamGridBuilder()
             .addGrid(rf.numTrees, [100, 200, 300])
             .addGrid(rf.maxDepth, [5, 10, 15])
             .addGrid(rf.maxBins, [32])
             .addGrid(rf.minInstancesPerNode, [1])
             .build())

evaluator = RegressionEvaluator(labelCol=label_col, predictionCol="prediction", metricName="rmse")

crossval = CrossValidator(estimator=rf,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=3,
                          parallelism=1)

cvModel = crossval.fit(train_data)
bestModel_rf = cvModel.bestModel

pred_rf = bestModel_rf.transform(valid_data)

rf_rmse, rf_r2 = evaluate(pred_rf, label_col)

print(f"Best Parameters:")
print(f"  - numTrees: {bestModel_rf.getNumTrees}")
print(f"  - maxDepth: {bestModel_rf.getMaxDepth}")
print(f"  - maxBins: {bestModel_rf.getMaxBins}")
print(f"  - minInstancesPerNode: {bestModel_rf.getMinInstancesPerNode}")
print(f"RMSE: {rf_rmse:.4f}")
print(f"R²: {rf_r2:.4f}")



Training Random Forest...


25/10/27 07:59:29 WARN DAGScheduler: Broadcasting large task binary with size 1065.9 KiB
25/10/27 07:59:29 WARN DAGScheduler: Broadcasting large task binary with size 1556.2 KiB
25/10/27 07:59:33 WARN DAGScheduler: Broadcasting large task binary with size 1065.9 KiB
25/10/27 07:59:33 WARN DAGScheduler: Broadcasting large task binary with size 1556.2 KiB
25/10/27 07:59:34 WARN DAGScheduler: Broadcasting large task binary with size 2.2 MiB
25/10/27 07:59:34 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB
25/10/27 07:59:35 WARN DAGScheduler: Broadcasting large task binary with size 4.6 MiB
25/10/27 07:59:36 WARN DAGScheduler: Broadcasting large task binary with size 6.3 MiB
25/10/27 07:59:38 WARN DAGScheduler: Broadcasting large task binary with size 8.3 MiB
25/10/27 07:59:42 WARN DAGScheduler: Broadcasting large task binary with size 1065.9 KiB
25/10/27 07:59:42 WARN DAGScheduler: Broadcasting large task binary with size 1556.2 KiB
25/10/27 07:59:43 WARN DAGScheduler:

RMSE = 181194.8568 | R² = 0.7811
Best Parameters:
  - numTrees: 300
  - maxDepth: <bound method _DecisionTreeParams.getMaxDepth of RandomForestRegressionModel: uid=RandomForestRegressor_89b53cc54add, numTrees=300, numFeatures=421>
  - maxBins: <bound method _DecisionTreeParams.getMaxBins of RandomForestRegressionModel: uid=RandomForestRegressor_89b53cc54add, numTrees=300, numFeatures=421>
  - minInstancesPerNode: <bound method _DecisionTreeParams.getMinInstancesPerNode of RandomForestRegressionModel: uid=RandomForestRegressor_89b53cc54add, numTrees=300, numFeatures=421>
RMSE: 181194.8568
R²: 0.7811


#### 4.4. XGBoost

In [18]:
print("\nTraining XGBoost...")

xgb = SparkXGBRegressor(
    features_col="features",
    label_col=label_col,
    objective="reg:squarederror",
    eval_metric="rmse",
    num_workers=1
)

paramGrid = (ParamGridBuilder()
             .addGrid(xgb.max_depth, [4, 6, 8])
             .addGrid(xgb.learning_rate, [0.05, 0.1, 0.2])
             .addGrid(xgb.subsample, [0.8, 1.0])
             .addGrid(xgb.colsample_bytree, [0.8, 1.0])
             .build())

evaluator = RegressionEvaluator(labelCol=label_col, predictionCol="prediction", metricName="rmse")

crossval = CrossValidator(estimator=xgb,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=3,
                          parallelism=2)

cvModel = crossval.fit(train_data)
bestModel_xgb = cvModel.bestModel

pred_xgb = bestModel_xgb.transform(valid_data)

xgb_rmse, xgb_r2 = evaluate(pred_xgb, label_col)

param_map = bestModel_xgb.extractParamMap()

print("\nBest Parameters:")
print(f"  - max_depth: {bestModel_xgb.getOrDefault('max_depth')}")
print(f"  - learning_rate: {bestModel_xgb.getOrDefault('learning_rate')}")
print(f"  - subsample: {bestModel_xgb.getOrDefault('subsample')}")
print(f"  - colsample_bytree: {bestModel_xgb.getOrDefault('colsample_bytree')}")

print(f"\nPerformance:")
print(f"  RMSE: {xgb_rmse:.4f}")
print(f"  R²: {xgb_r2:.4f}")



Training XGBoost...


2025-10-27 08:14:07,249 INFO XGBoost-PySpark: _fit Running xgboost-2.0.3 on 1 workers with
	booster params: {'colsample_bytree': 1.0, 'device': 'cpu', 'learning_rate': 0.05, 'max_depth': 4, 'objective': 'reg:squarederror', 'subsample': 0.8, 'eval_metric': 'rmse', 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}
2025-10-27 08:14:07,250 INFO XGBoost-PySpark: _fit Running xgboost-2.0.3 on 1 workers with
	booster params: {'colsample_bytree': 0.8, 'device': 'cpu', 'learning_rate': 0.05, 'max_depth': 4, 'objective': 'reg:squarederror', 'subsample': 0.8, 'eval_metric': 'rmse', 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}
[08:14:11] task 0 got new rank 0[08:14:11] task 0 got new rank 0    (0 + 1) / 1]

2025-10-27 08:14:13,133 INFO XGBoost-PySpark: _fit Finished xgboost training!   
2025-10-27 08:14:13,134 INFO XGBoost-

RMSE = 100819.6583 | R² = 0.9322

Best Parameters:
  - max_depth: 8
  - learning_rate: 0.1
  - subsample: 0.8
  - colsample_bytree: 0.8

Performance:
  RMSE: 100819.6583
  R²: 0.9322


INFO:XGBoost-PySpark:Do the inference on the CPUs


#### 4.5. Đánh giá mô hình

In [19]:
# Đánh giá tất cả mô hình trên test
print("Đánh giá tất cả mô hình trên TEST SET:")
print("="*50)

# 1. Linear Regression 
print("\n1. Linear Regression:")
pred_lr = bestModel_lr.transform(test_data)
lr_rmse, lr_r2 = evaluate(pred_lr, label_col)

# 2. Decision Tree
print("\n2. Decision Tree :")
pred_dt_test = bestModel_dt.transform(test_data)
dt_rmse, dt_r2 = evaluate(pred_dt_test, label_col)

# 3. Random Forest 
print("\n3. Random Forest :")
pred_rf_test = bestModel_rf.transform(test_data)
rf_rmse, rf_r2 = evaluate(pred_rf_test, label_col)

# 4. XGBoost 
print("\n4. XGBoost:")
pred_xgb = bestModel_xgb.transform(test_data)
xgb_rmse, xgb_r2 = evaluate(pred_xgb, label_col)

Đánh giá tất cả mô hình trên TEST SET:

1. Linear Regression:
RMSE = 208877.4408 | R² = 0.7097

2. Decision Tree :
RMSE = 185647.3547 | R² = 0.7707

3. Random Forest :
RMSE = 177980.4616 | R² = 0.7892

4. XGBoost:


25/10/27 08:19:06 WARN DAGScheduler: Broadcasting large task binary with size 1023.1 KiB
INFO:XGBoost-PySpark:Do the inference on the CPUs
25/10/27 08:19:08 WARN DAGScheduler: Broadcasting large task binary with size 1023.1 KiB


RMSE = 97728.1342 | R² = 0.9365


INFO:XGBoost-PySpark:Do the inference on the CPUs


#### Tóm tắt kết quả các mô hình

Dưới đây là kết quả so sánh hiệu suất của 4 mô hình dự đoán giá nhà:

In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Tạo bảng tóm tắt kết quả từ các biến đã tính toán
results_data = {
    'Mô hình': ['Linear Regression', 'Decision Tree', 'Random Forest', 'XGBoost'],
    'RMSE': [lr_rmse, dt_rmse, rf_rmse, xgb_rmse],
    'R²': [lr_r2, dt_r2, rf_r2, xgb_r2]
}

results_df = pd.DataFrame(results_data)
print("BẢNG TỔNG KẾT KẾT QUẢ CÁC MÔ HÌNH")
print("="*60)
print(results_df.to_string(index=False))
print("="*60)

BẢNG TỔNG KẾT KẾT QUẢ CÁC MÔ HÌNH
          Mô hình          RMSE       R²
Linear Regression 208877.440828 0.709720
    Decision Tree 185647.354670 0.770696
    Random Forest 177980.461619 0.789244
          XGBoost  97728.134198 0.936456
